In [20]:
# For downloading historical market data:
import yfinance as yf

# For data manipulation:
import pandas as pd

# For working with dates and times:
from datetime import datetime, timedelta

# For timezone support:
import pytz

# <font color='pink'>Utility Functions</font>


Removing seconds from datetime strings in a Dataframe.

In [21]:
def removeSeconds(df):
    modified_dates=[]
    for date in df['Date']:

        format = '%Y-%m-%d %H:%M:%S%z'

        # Converting stirng to datetime object using the above format:
        date = datetime.strptime(date, format)

        # Removing seconds from the datetime object:
        modified_date = date.replace(second=0)
        
        modified_dates.append(modified_date)
    df['Date']=modified_dates
    return df

Converting datetime values in a DataFrame to a specific timezone.

In [22]:
def convertTimezone(timezone,df,name):

    # The target timezone:
    target_tz = pytz.timezone(timezone)

    # If we're using DataFrame index:
    if df.index is not None:
        # Converting the datetime to the target timezone:
        df.index = pd.to_datetime(df.index, utc=True).tz_convert(timezone)

    # If we're not using DataFrame index:
    else:
        # Converting the datetime to the target timezone:
        df[name] = pd.to_datetime(df[name],utc=True).dt.tz_convert(target_tz)
    return df

# NOTES:
    # 'utc=True' for ensuring that all datetime values are in UTC before any operations
    # ... to avoid issues related to timezone conversions

Retrieving stock prices for a specified date range using Yahoo Finance API 'yfinance'.

In [23]:
def getStockPrices(start_date,end_date,ticker):

    # Converting start and end date to datetime objects:
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")

    # Interval for stock price data retrieval (1 minute):
    interval = "1m"                     

    # Assigning start_date to current_date:
    current_date = start_date
    
    # List that will contain all retrieved stock price data:
    data_list = []  

    todays_date = datetime.now()

    while current_date < end_date:

        # Since we're downloading stock price data in chunks of up to 7 days to avoid API limitations,
        # ... we must calculate the remaining days at each iteration:
        remaining_days = (end_date - current_date).days
        
        # If remaining_days < 7, we download the data until end_date
        if remaining_days < 7:
            print(f"getting dates for {current_date}->{next_date}")
            data = yf.download(tickers=ticker, start=current_date, end=end_date, interval=interval)

            # We must convert the data to a specified timezone:
            data = convertTimezone('UTC', data, 'Datetime')
            
            data_list.append(data)

            # Setting current_date to end_date to exit the loop
            current_date = end_date

        # If remaining_days>=7, we download data for next 7 days:
        else:
            next_date = current_date + timedelta(days=7)

            print(f"getting dates for {current_date}->{next_date}")
            data = yf.download(tickers=ticker, start=current_date, end=next_date, interval=interval)

            # We must convert the data to a specified timezone:
            data = convertTimezone('UTC', data, 'Datetime')
            
            data_list.append(data)

            # Updating current_date for the next iteration:
            current_date = next_date

    final_data = pd.concat(data_list)
    final_data.reset_index(inplace=True)
    if 'index' in final_data.columns:  
        final_data.rename(columns={'index': 'Datetime'}, inplace=True)  
    return final_data

# Notes:
    # We must convert the data to a specified timezone, 
    # ... to ensure that all datetime values in the dataset are in the same timezone,
    # ... for consistency and comparison purposes.


Adding stock prices to a DataFrame based on matching dates

In [24]:
def addStockPrices(stockPrices,df):

    # Ensuring that both datasets have their datetime columns properly formatted as strings,
    # ... for accurate matching.
    stockPrices['Datetime'] = pd.to_datetime(stockPrices['Datetime'],utc=True)
    stockPrices['Datetime'] = stockPrices['Datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')

    df['Date'] = pd.to_datetime(df['Date'],utc=True)
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')

    # This list will store retrieved stock prices:
    stock_prices = []

    for index, row in df.iterrows():

        publication_date = row['Date']
       
        # If the publication date exists in teh stockPrices DataFrame,
        # ... we retrieve the corresponding stock price from stockPrices DataFrame
        if publication_date in stockPrices['Datetime'].values:
            stock_price = stockPrices.loc[stockPrices['Datetime'] == publication_date, 'Adj Close'].values[0]
            stock_prices.append(stock_price)
        else:
            stock_prices.append(None)  
    df['stock_price'] = stock_prices
    df= df.dropna(subset=['stock_price'])
    return df

Adding stock prices after a specified time period to a DataFrame based on publication dates.

In [25]:
def addStockPricesAfter(stockPrices, df, time):
    # a copy of the DataFrame to avoid modifying the original
    df_copy = df.copy()

    # Ensuring that both datasets have their datetime columns properly formatted as strings,
    # ... for accurate matching.
    stockPrices['Datetime'] = pd.to_datetime(stockPrices['Datetime'], utc=True)
    stockPrices['Datetime'] = stockPrices['Datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')

    df_copy['Date'] = pd.to_datetime(df_copy['Date'], utc=True)

    # Adding the specified time period to the publication dates
    df_copy['Date'] += timedelta(minutes=time)

    df_copy['Date'] = df_copy['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')

    # This list will store retrieved stock prices:
    stock_prices = []

    for index, row in df_copy.iterrows():
        publication_date = row['Date']

        # If the publication date exists in the stockPrices DataFrame,
        # ... we retrieve the corresponding stock price from stockPrices DataFrame
        if publication_date in stockPrices['Datetime'].values:
            stock_price = stockPrices.loc[stockPrices['Datetime'] == publication_date, 'Adj Close'].values[0]
            stock_prices.append(stock_price)
        else:
            stock_prices.append(None)

    df_copy[f'stock_price_after_{time}_mins'] = stock_prices
    df_copy = df_copy.dropna(subset=[f'stock_price_after_{time}_mins'])

    # Calculating the change percentages
    change_percentages=[]

    for index, row in df_copy.iterrows():
        change_percentage=((row[f'stock_price_after_{time}_mins'] - row['stock_price'] )/ row['stock_price']) * 100
        change_percentages.append(change_percentage)

    df_copy['change_percentage']=change_percentages
    return df_copy

Labeling the Data based on stock price changes after a certain period of time.

In [26]:
def label(df,down,up):
    # List to store labels indicating stock price changes
    labels=[]

    down_threshold = down
    up_threshold = up  
    
    for index, row in df.iterrows():

        # Labeling the data:
        if row['change_percentage']>up_threshold:
            labels.append(1)
        elif row['change_percentage']<down_threshold:
            labels.append(-1)
        else:
            labels.append(0)

    df[f'label']=labels
    return df

Creating a CSV file

In [27]:
def turnToCSV(df,name):
    df.to_csv(fr'..\data\{name}.csv',index=False)

# <font color='pink'>Main Program</font>

Reading our News dataset tha contains all of the News articles and their publication dates

In [28]:
News=pd.read_csv(r'..\data\News.csv')
News.head(1)


,Date,article_title,article,source_name,source_link,ticker_symbol,Positive,Negative,Neutral,Compound,Sentiment
0,2024-05-02 16:20:34+00:00,25 Most Profitable Companies in the US,"In this piece, we will take a look at the 25 m...",Yahoo Finance,https://finance.yahoo.com/news/25-most-profita...,AAPL,0.107,0.025,0.869,0.9992,Positive


Creating the stockPrices dataset

In [29]:
# Getting the start date from the News dataset and the end date:
start_date = News['Date'].min().split()[0] 
end_date =  News['Date'].max().split()[0]  
print("Starting Date:",start_date)
print("Ending Date:",end_date)

Starting Date: 2024-03-05
Ending Date: 2024-05-02


In [30]:
# Creating a CSV file with all the stock prices corresponding to these dates:
#stockPrices=getStockPrices(start_date,end_date,'AAPL')

try:
    existing_data = pd.read_csv(r"..\data\StockPrices.csv")
except FileNotFoundError:
    existing_data = pd.DataFrame()

new_data=getStockPrices(start_date,end_date,'AAPL')

# Appending new data to existing DataFrame
stockPrices = pd.concat([existing_data, new_data])
stockPrices.drop_duplicates(subset='Datetime', keep='first', inplace=True)

turnToCSV(stockPrices,'stockPrices')
stockPrices

getting dates for 2024-03-05 00:00:00->2024-03-12 00:00:00
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AAPL: 1m data not available for startTime=1709589600 and endTime=1710194400. The requested range must be within the last 30 days.
getting dates for 2024-03-12 00:00:00->2024-03-19 00:00:00
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AAPL: 1m data not available for startTime=1710194400 and endTime=1710799200. The requested range must be within the last 30 days.
getting dates for 2024-03-19 00:00:00->2024-03-26 00:00:00
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AAPL: 1m data not available for startTime=1710799200 and endTime=1711404000. The requested range must be within the last 30 days.
getting dates for 2024-03-26 00:00:00->2024-04-02 00:00:00
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AAPL: 1m d

,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2024-03-05 14:30:00+00:00,171.000000,171.000000,170.610001,170.919998,170.919998,5958861.0
1,2024-03-05 14:31:00+00:00,171.020004,171.074997,171.020004,171.059998,171.059998,926341.0
2,2024-03-05 14:32:00+00:00,171.335999,171.339996,171.270004,171.300003,171.300003,703908.0
3,2024-03-05 14:33:00+00:00,171.929993,171.970001,171.875000,171.895004,171.895004,994237.0
4,2024-03-05 14:34:00+00:00,171.625000,171.740005,171.580002,171.610001,171.610001,626087.0
...,...,...,...,...,...,...,...
6615,2024-05-01 19:55:00+00:00,170.104996,170.570007,169.804993,169.929993,169.929993,551196.0
6616,2024-05-01 19:56:00+00:00,169.919998,169.958206,169.710007,169.800003,169.800003,343206.0
6617,2024-05-01 19:57:00+00:00,169.789993,169.809998,169.554993,169.679993,169.679993,389352.0
6618,2024-05-01 19:58:00+00:00,169.690002,169.799896,169.520004,169.549698,169.549698,636511.0


Getting the stock prices corresponding to the publication date and removing seconds from the date

In [31]:
# First, we need to remove the seconds from the dates since 'yfinance operates on minute-based intervals:
News=removeSeconds(News)

# Adding the stock prices corresponding to the publication date:
News=addStockPrices(stockPrices,News)
turnToCSV(News,'NewsWithStockPrices')

Getting the stock prices after a certain amount of time from the publication date and Labeling the data.<br>
Created 3 new datasets, for clarity, each one for a specified time delay.

In [32]:
# Adding stock prices after 30 minutes of the publication date:
after_30_mins=addStockPricesAfter(stockPrices,News,30)
after_30_mins=label(after_30_mins,-0.1,0.1)
after_30_mins.to_csv(r'..\data\stockPrices\stock_prices_after_30_mins.csv',index=False)

# Adding stock prices after 6 hours (360 minutes) of the publication date:
after_6_hours=addStockPricesAfter(stockPrices,News,120)
after_6_hours=label(after_6_hours,-0.2,0.2)
after_6_hours.to_csv(r'..\data\stockPrices\stock_prices_after_6_hours.csv',index=False)

# Adding stock prices after 1 day (1440 minutes) of the publication date:
after_1_day=addStockPricesAfter(stockPrices,News,1440)
after_1_day=label(after_1_day,-0.4,0.4)
after_1_day.to_csv(r'..\data\stockPrices\stock_prices_after_1_day.csv',index=False)


In [33]:
after_30_mins.describe()

,Positive,Negative,Neutral,Compound,stock_price,stock_price_after_30_mins,change_percentage,label
count,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000
mean,0.105615,0.044297,0.850078,0.555855,171.058031,171.080873,0.014202,0.050802
std,0.069791,0.041381,0.070424,0.653821,2.636212,2.581542,0.269135,0.788719
min,0.000000,0.000000,0.424000,-0.997700,164.362396,164.964996,-1.212901,-1.000000
25%,0.066000,0.015000,0.821250,0.328950,169.417503,169.466999,-0.143791,-1.000000
50%,0.096500,0.036500,0.857000,0.949650,170.917549,171.039993,0.017019,0.000000
75%,0.126750,0.066000,0.890750,0.995825,172.472496,172.468826,0.165605,1.000000
max,0.576000,0.256000,1.000000,1.000000,178.379898,178.360001,1.012974,1.000000


In [34]:
after_6_hours.describe()

,Positive,Negative,Neutral,Compound,stock_price,stock_price_after_120_mins,change_percentage,label
count,307.000000,307.000000,307.000000,307.000000,307.000000,307.000000,307.000000,307.000000
mean,0.103231,0.043990,0.852785,0.564232,171.052737,170.986315,-0.036443,0.026059
std,0.067070,0.042592,0.069260,0.637993,2.671134,2.562128,0.537232,0.831943
min,0.000000,0.000000,0.424000,-0.997700,164.362396,164.119995,-2.334167,-1.000000
25%,0.065500,0.015000,0.821500,0.295700,169.324997,169.387505,-0.381616,-1.000000
50%,0.096000,0.035000,0.860000,0.947200,170.908401,171.130096,-0.006364,0.000000
75%,0.126500,0.063000,0.891500,0.994800,172.419998,172.319000,0.286424,1.000000
max,0.576000,0.256000,1.000000,1.000000,177.598694,178.289993,2.181137,1.000000


In [35]:
after_1_day.describe()

,Positive,Negative,Neutral,Compound,stock_price,stock_price_after_1440_mins,change_percentage,label
count,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000
mean,0.108257,0.043042,0.848675,0.551523,171.186125,171.299184,0.073522,0.003215
std,0.073812,0.040612,0.074413,0.656464,2.637493,2.485023,1.115719,0.801202
min,0.000000,0.000000,0.424000,-0.997700,165.510300,165.009995,-4.201370,-1.000000
25%,0.067000,0.011500,0.820500,0.295700,169.230003,169.570595,-0.575441,-1.000000
50%,0.097000,0.036000,0.857000,0.942300,170.684998,171.145004,0.082906,0.000000
75%,0.133500,0.063500,0.891000,0.995250,172.527496,172.739998,0.681639,1.000000
max,0.576000,0.256000,1.000000,1.000000,178.379898,177.500000,5.104213,1.000000


In [36]:
after_30_mins.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374 entries, 16 to 1012
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       374 non-null    object 
 1   article_title              374 non-null    object 
 2   article                    374 non-null    object 
 3   source_name                374 non-null    object 
 4   source_link                374 non-null    object 
 5   ticker_symbol              374 non-null    object 
 6   Positive                   374 non-null    float64
 7   Negative                   374 non-null    float64
 8   Neutral                    374 non-null    float64
 9   Compound                   374 non-null    float64
 10  Sentiment                  374 non-null    object 
 11  stock_price                374 non-null    float64
 12  stock_price_after_30_mins  374 non-null    float64
 13  change_percentage          374 non-null    float

In [37]:
after_6_hours.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307 entries, 16 to 1012
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Date                        307 non-null    object 
 1   article_title               307 non-null    object 
 2   article                     307 non-null    object 
 3   source_name                 307 non-null    object 
 4   source_link                 307 non-null    object 
 5   ticker_symbol               307 non-null    object 
 6   Positive                    307 non-null    float64
 7   Negative                    307 non-null    float64
 8   Neutral                     307 non-null    float64
 9   Compound                    307 non-null    float64
 10  Sentiment                   307 non-null    object 
 11  stock_price                 307 non-null    float64
 12  stock_price_after_120_mins  307 non-null    float64
 13  change_percentage           307 n

In [38]:
after_1_day.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 311 entries, 30 to 1012
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Date                         311 non-null    object 
 1   article_title                311 non-null    object 
 2   article                      311 non-null    object 
 3   source_name                  311 non-null    object 
 4   source_link                  311 non-null    object 
 5   ticker_symbol                311 non-null    object 
 6   Positive                     311 non-null    float64
 7   Negative                     311 non-null    float64
 8   Neutral                      311 non-null    float64
 9   Compound                     311 non-null    float64
 10  Sentiment                    311 non-null    object 
 11  stock_price                  311 non-null    float64
 12  stock_price_after_1440_mins  311 non-null    float64
 13  change_percentage 